# Integrated Study Assistant with Web Browsing

This notebook combines the study assistant with web browsing capabilities, allowing the assistant to:
1. Retrieve information from local document storage (course materials)
2. Search and scrape the web for recent or external information
3. Provide direct responses for general questions

In [1]:
# Import required libraries
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_chroma import Chroma
from langchain_core.tools import tool
from typing_extensions import TypedDict
from typing import List, Dict, Any, Tuple, Optional, Union, Literal
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.types import Command
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from pydantic import BaseModel, Field
from retriever import contextual_embedding_bm25_retriever_rerank
from web_browsing_agent import web_search, web_scrape, browsing_agent
from dotenv import load_dotenv
import os
import re

load_dotenv()

c:\Users\gokul\anaconda3\envs\langgraphenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

## Setup Components

Initialize memory, configuration, and tools

In [2]:
# Initialize memory saver for graph persistence
memory = MemorySaver()
config = {'configurable': {'thread_id': "1"}}

# Enable debug logging to track paths through the graph
DEBUG = True  # Set to False to disable debug prints

def debug_print(message):
    if DEBUG:
        print(f"[DEBUG] {message}")

In [3]:
# Define the contextual retriever tool for local document search
@tool
def contexual_retriever(query: str) -> List[str]:
    """
    Retrieve all relevant documents with similarity threshold filtering (>50%).

    Provide very large,or large detailed answers if needed.
    The agent should try to return comprehensive context rather than short responses.

    Args:
        query (str): Search query string.
        
    Returns:
        List[Union[str, Document]]: All Document objects with relevance_score > 0.5,
                                    sorted by relevance_score in descending order,
                                    or ["Answer is not in Database"] if no matches.
    """
    debug_print(f"Retrieving documents for query: {query}")
    # Get all results from the retriever using the input query as is.
    results = contextual_embedding_bm25_retriever_rerank.invoke(query)
    
    # Filter results that have a relevance_score > 0.5 (i.e., >50%).
    filtered_results = [
        doc for doc in results 
        if doc.metadata.get('relevance_score', 0.0) > 0.5
    ]
    
    # Sort the filtered results by relevance_score in descending order.
    filtered_results.sort(
        key=lambda x: x.metadata.get('relevance_score', 0.0),
        reverse=True
    )

    debug_print(f"Found {len(filtered_results)} relevant documents")
    if not filtered_results:
        return ["Answer is not in Database"]
    
    return filtered_results

## Initialize Models and Agents

Set up the LLM models and create the retrieval agent

In [4]:
# Initialize LLM models
llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash', temperature=0)
query_llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash', temperature=0)
study_assistant_model = ChatGoogleGenerativeAI(model='gemini-2.0-flash', temperature=0)

In [5]:
# Create retrieval agent for local documents
retrieve_agent = create_react_agent(llm, tools=[contexual_retriever])

## Define Router and State Classes

Configure the router to direct queries to the appropriate path

In [6]:
# Updated Router class with improved descriptions for routing clarity
class Router(BaseModel):
    goto: Literal['query_creator', 'study_assistant', 'web_browser_query'] = Field(
        description=(
            "Given a user query to you, You choose the next node to go to.\n"
            "Use 'query_creator' if the user's request is specifically about study materials or documents in our database.\n"
            "This includes ANY questions related to the following course subjects: DeepLearning AI, Edge Computing, "
            "LargeLanguageModels, Security Essentials in Applied AI, and Distributed DataProcessing with Spark. "
            "The query_creator node will generate a query for retrieval from the stored vectorbase to perform RAG.\n"
            "Use 'web_browser_query' if the user's request requires searching for recent information, external data, or topics "
            "not likely to be in the course materials. This includes current events, latest trends, any information "
            "that would need internet access, or if the user explicitly asks to scrape or browse a specific URL.\n"
            "Use 'study_assistant' only for direct responses that don't require retrieving specific information "
            "from either the course materials or the web."
        )
    )

In [7]:
# Setup structured output model for routing
model = ChatGoogleGenerativeAI(model='gemini-2.0-flash', temperature=0)
model_structured = model.with_structured_output(Router)

In [8]:
# System instructions
query_instruct = SystemMessage(content="""You need to create a query, which will be passed to a contextual retriever tool.
                               The retriever is connected to a vectorsore to do similarity search on the documents in the database. 
                               The retriever will return the most relevant documents based on the query. 
                               The documents will be used to generate a response to your question.
                               Make sure to provide a query that only beautifies the user's question and does not add any new information, or change its meaning.
                               Please provide a query to search for relevant documents in the database.""")

web_search_instruct = SystemMessage(content="""You need to create a search query for the web browsing agent.
                                    This query will be used to search the internet for relevant information or scrape a specific URL.
                                    If the user's query contains a specific URL to scrape, preserve that exact URL in your query.
                                    If it's a general search request, make it clear, concise, and search-friendly.
                                    Make sure your query captures the main points while being suitable for web search or URL scraping.
                                    DO NOT modify URLs provided by the user - pass them through exactly as given.""")

study_instruct = SystemMessage(content="""You are a study assistant, you need to provide a detailed answer to the user's question.
                               You can provide detailed answers to the user's questions based on your knowledge.""")

study_instruct_with_response = SystemMessage(content="""You are a study assistant, you need to provide a detailed answer to the user's question.
                                             You are given a response from a contextual retriever tool or web browsing tool and also the User's question.
                                             Use the retrieved answers and the user's question to provide a detailed response.
                                             If the retrieved answers are not sufficient, you can provide your own answers.
                                             If the retrieved answers are not relevant, you can provide your own answers.""")

In [9]:
# Define graph state
class State(MessagesState):
    query: str
    response: str
    goto: str
    search_results: Optional[List[Dict[str, str]]] = None
    web_content: Optional[str] = None
    debug_info: Optional[Dict[str, Any]] = None

## Define Node Functions

Create the node functions that will process the state in the graph

In [10]:
# Node functions
def router(state: State) -> Command[Literal["query_creator", "study_assistant", "web_browser_query"]]:
    debug_print(f"Router received query: {state['messages'][-1].content}")
    answer = model_structured.invoke(state["messages"][-1].content)
    response = getattr(answer, "goto")
    goto = response
    debug_print(f"Router decision: {goto}")
    debug_info = state.get('debug_info', {}) or {}
    debug_info['router_decision'] = goto
    return Command(goto=goto, update={"goto": goto, "messages": state["messages"], "debug_info": debug_info})

In [11]:
def query_creator(state: State) -> State:
    debug_print("Processing in query_creator node")
    response = query_llm.invoke([query_instruct] + [state['messages'][-1].content])
    state['query'] = response.content
    debug_print(f"Created query: {state['query']}")
    return state

In [12]:
def retriever_agent(state: State) -> State:
    debug_print("Processing in retriever_agent node")
    response = retrieve_agent.invoke({"messages": HumanMessage(content=state['query'])})
    state['response'] = response['messages'][-1].content
    debug_print(f"Retriever response length: {len(state['response'])}")
    return state

In [13]:
def web_browser_query(state: State) -> State:
    debug_print("Processing in web_browser_query node")
    # Create a search query from the user's question
    response = query_llm.invoke([web_search_instruct] + [state['messages'][-1].content])
    state['query'] = response.content
    debug_print(f"Created web query: {state['query']}")
    return state

In [14]:
def browser_node(state: State) -> State:
    """Process user queries that require web browsing capabilities.
    
    This function can handle both direct URL scraping requests and general web search queries.
    """
    debug_print("Processing in browser_node")
    query = state['query']
    
    # Check if the query contains a URL to scrape
    url_indicators = ["http://", "https://", "www.", ".com", ".org", ".net", ".edu", ".gov", "scrape"]
    contains_url = any(indicator in query.lower() for indicator in url_indicators)
    
    debug_info = state.get('debug_info', {}) or {}
    debug_info['contains_url'] = contains_url
    state['debug_info'] = debug_info
    
    if contains_url:
        debug_print("URL detected in query - attempting to scrape")
        # Extract URL from the query if possible
        url_match = re.search(r'https?://[^\s]+', query)
        
        if url_match:
            url = url_match.group(0)
            debug_print(f"Scraping URL: {url}")
            # Use web_scrape directly from the imported module
            try:
                scraped_content = web_scrape(url)
                state['web_content'] = scraped_content
                state['response'] = f"I've scraped the content from {url}. Here's what I found:\n\n{scraped_content[:2000]}...\n\n(Content truncated for readability)"
                debug_info['scraping_success'] = True
                debug_info['scraped_url'] = url
            except Exception as e:
                debug_print(f"Error scraping URL: {str(e)}")
                state['response'] = f"I encountered an error while trying to scrape {url}: {str(e)}"
                debug_info['scraping_success'] = False
                debug_info['error'] = str(e)
        else:
            debug_print("URL format unclear - using browsing agent")
            # If URL format is unclear, use the browsing agent which can handle both search and scraping
            response = browsing_agent.invoke({"messages": HumanMessage(content=query)})
            state['response'] = response['messages'][-1].content
            debug_info['used_browsing_agent'] = True
    else:
        debug_print("No URL detected - using browsing agent for general search")
        # For general search queries, use the browsing agent
        response = browsing_agent.invoke({"messages": HumanMessage(content=query)})
        search_results = response['messages'][-1].content
        state['response'] = search_results
        debug_info['used_browsing_agent'] = True
    
    state['debug_info'] = debug_info
    debug_print(f"Browser node response length: {len(state['response'])}")
    return state

In [15]:
def study_assistant(state: State) -> State:
    debug_print(f"Processing in study_assistant node (goto={state['goto']})")
    if state['goto'] == 'query_creator':
        debug_print("Using document retrieval response")
        response = study_assistant_model.invoke([study_instruct_with_response] + [state['response']] + [state['messages'][-1].content])
        state['response'] = None
        state['messages'] = state['messages'] + [response]
    elif state['goto'] == 'web_browser_query':
        debug_print("Using web browsing response")
        response = study_assistant_model.invoke([study_instruct_with_response] + [state['response']] + [state['messages'][-1].content])
        state['response'] = None
        state['search_results'] = None
        state['web_content'] = None
        state['messages'] = state['messages'] + [response]
    else:
        debug_print("Using direct response from knowledge")
        response = study_assistant_model.invoke([study_instruct] + state['messages'])
        state['messages'] = state['messages'] + [response]
    return state

## Build and Compile the Graph

Construct the graph with all nodes and edge connections

In [16]:
# Build the graph
graph_builder = StateGraph(State)
graph_builder.add_node("router", router)
graph_builder.add_node("study_assistant", study_assistant)
graph_builder.add_node("query_creator", query_creator)
graph_builder.add_node("retriever_agent", retriever_agent)
graph_builder.add_node("web_browser_query", web_browser_query)
graph_builder.add_node("browser_node", browser_node)

In [17]:
# Define the edges - using conditional edges for routing
graph_builder.add_edge(START, "router")
# Use a lambda function that returns the goto value directly
graph_builder.add_conditional_edges(
    "router",
    lambda state: state["goto"],
    {
        "query_creator": "query_creator",
        "study_assistant": "study_assistant",
        "web_browser_query": "web_browser_query"
    }
)

In [18]:
# Complete the graph with remaining edges
graph_builder.add_edge("query_creator", "retriever_agent")
graph_builder.add_edge("retriever_agent", "study_assistant")
graph_builder.add_edge("web_browser_query", "browser_node")
graph_builder.add_edge("browser_node", "study_assistant")
graph_builder.add_edge("study_assistant", END)

In [19]:
# Compile the graph
graph = graph_builder.compile(checkpointer=memory)

## Testing the Integrated Study Assistant

Let's test our assistant with queries that should use different paths through the graph.

### Test 1: URL Scraping Capability
Testing direct URL scraping functionality

In [21]:
print("Example 1: URL Scraping (should use web browsing with direct scraping)")
response = graph.invoke({'messages':[HumanMessage(content="https://en.wikipedia.org/wiki/Artificial_intelligence scrape this page about AI")]},config=config)

Example 1: URL Scraping (should use web browsing with direct scraping)
[DEBUG] Router received query: https://en.wikipedia.org/wiki/Artificial_intelligence scrape this page about AI
[DEBUG] Router received query: https://en.wikipedia.org/wiki/Artificial_intelligence scrape this page about AI
[DEBUG] Router decision: web_browser_query
[DEBUG] Processing in web_browser_query node
[DEBUG] Router decision: web_browser_query
[DEBUG] Processing in web_browser_query node
[DEBUG] Created web query: scrape: https://en.wikipedia.org/wiki/Artificial_intelligence
[DEBUG] Processing in browser_node
[DEBUG] URL detected in query - attempting to scrape
[DEBUG] Scraping URL: https://en.wikipedia.org/wiki/Artificial_intelligence
[DEBUG] Created web query: scrape: https://en.wikipedia.org/wiki/Artificial_intelligence
[DEBUG] Processing in browser_node
[DEBUG] URL detected in query - attempting to scrape
[DEBUG] Scraping URL: https://en.wikipedia.org/wiki/Artificial_intelligence


C:\Users\gokul\AppData\Local\Temp\ipykernel_20804\588007152.py:27: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  scraped_content = web_scrape(url)


[DEBUG] Browser node response length: 2147
[DEBUG] Processing in study_assistant node (goto=web_browser_query)
[DEBUG] Using web browsing response


In [22]:
print("\nResponse:")
for msg in response['messages']:
    if msg.type == "ai":
        print(msg.content[:500] + "...")

# Print debug info if available
if DEBUG and 'debug_info' in response:
    print("\nDebug Info:")
    for k, v in response.get('debug_info', {}).items():
        print(f"{k}: {v}")


Response:
I have scraped the Wikipedia page on Artificial Intelligence and extracted the table of contents. The contents include:

1.  Goals
    *   Reasoning and problem-solving
    *   Knowledge representation
    *   Planning and decision-making
    *   Learning
    *   Natural language processing
    *   Perception
    *   Social intelligence
    *   General intelligence
2.  Techniques
    *   Search and optimization
        *   State space search
        *   Local search
    *   Logic
    *   Probabi...

Debug Info:
router_decision: web_browser_query
contains_url: True
scraping_success: True
scraped_url: https://en.wikipedia.org/wiki/Artificial_intelligence


### Test 2: Course Material Retrieval
Testing retrieval from local document store

In [23]:
print("Example 2: Query about Edge Computing (should use local documents)")
response = graph.invoke({"messages": [HumanMessage(content="What is edge computing architecture?")]}, config=config)

Example 2: Query about Edge Computing (should use local documents)
[DEBUG] Router received query: What is edge computing architecture?
[DEBUG] Router decision: query_creator
[DEBUG] Processing in query_creator node
[DEBUG] Router decision: query_creator
[DEBUG] Processing in query_creator node
[DEBUG] Created query: What is the architecture of edge computing?
[DEBUG] Processing in retriever_agent node
[DEBUG] Created query: What is the architecture of edge computing?
[DEBUG] Processing in retriever_agent node
[DEBUG] Retrieving documents for query: architecture of edge computing
[DEBUG] Retrieving documents for query: architecture of edge computing
[DEBUG] Found 10 relevant documents
[DEBUG] Found 10 relevant documents
[DEBUG] Retriever response length: 2193
[DEBUG] Processing in study_assistant node (goto=query_creator)
[DEBUG] Using document retrieval response
[DEBUG] Retriever response length: 2193
[DEBUG] Processing in study_assistant node (goto=query_creator)
[DEBUG] Using documen

In [24]:
print("\nResponse:")
for msg in response['messages']:
    if msg.type == "ai":
        print(msg.content[:500] + "...")

# Print debug info if available
if DEBUG and 'debug_info' in response:
    print("\nDebug Info:")
    for k, v in response.get('debug_info', {}).items():
        print(f"{k}: {v}")


Response:
I have scraped the Wikipedia page on Artificial Intelligence and extracted the table of contents. The contents include:

1.  Goals
    *   Reasoning and problem-solving
    *   Knowledge representation
    *   Planning and decision-making
    *   Learning
    *   Natural language processing
    *   Perception
    *   Social intelligence
    *   General intelligence
2.  Techniques
    *   Search and optimization
        *   State space search
        *   Local search
    *   Logic
    *   Probabi...
Edge computing architecture is a distributed computing paradigm that brings computation and data storage closer to the data source, reducing latency, saving bandwidth, and improving reliability. It's built upon several key principles and incorporates a combination of hardware and software components.

Here's a breakdown of the key aspects:

*   **Core Principles:**
    *   **Proximity:** Computation happens near the data source (e.g., sensors, devices).
    *   **Real-time Process

### Test 3: Web Search Capability
Testing web search for recent/external information

In [25]:
print("Example 3: Query about recent news (should use web browsing)")
response = graph.invoke({"messages": [HumanMessage(content="What are the latest trends in AI for 2025?")]}, config=config)

Example 3: Query about recent news (should use web browsing)
[DEBUG] Router received query: What are the latest trends in AI for 2025?
[DEBUG] Router decision: web_browser_query
[DEBUG] Processing in web_browser_query node
[DEBUG] Router decision: web_browser_query
[DEBUG] Processing in web_browser_query node
[DEBUG] Created web query: Latest AI trends 2025
[DEBUG] Processing in browser_node
[DEBUG] No URL detected - using browsing agent for general search
[DEBUG] Created web query: Latest AI trends 2025
[DEBUG] Processing in browser_node
[DEBUG] No URL detected - using browsing agent for general search
[DEBUG] Browser node response length: 213
[DEBUG] Processing in study_assistant node (goto=web_browser_query)
[DEBUG] Using web browsing response
[DEBUG] Browser node response length: 213
[DEBUG] Processing in study_assistant node (goto=web_browser_query)
[DEBUG] Using web browsing response


In [26]:
print("\nResponse:")
for msg in response['messages']:
    if msg.type == "ai":
        print(msg.content[:500] + "...")

# Print debug info if available
if DEBUG and 'debug_info' in response:
    print("\nDebug Info:")
    for k, v in response.get('debug_info', {}).items():
        print(f"{k}: {v}")


Response:
I have scraped the Wikipedia page on Artificial Intelligence and extracted the table of contents. The contents include:

1.  Goals
    *   Reasoning and problem-solving
    *   Knowledge representation
    *   Planning and decision-making
    *   Learning
    *   Natural language processing
    *   Perception
    *   Social intelligence
    *   General intelligence
2.  Techniques
    *   Search and optimization
        *   State space search
        *   Local search
    *   Logic
    *   Probabi...
Edge computing architecture is a distributed computing paradigm that brings computation and data storage closer to the data source, reducing latency, saving bandwidth, and improving reliability. It's built upon several key principles and incorporates a combination of hardware and software components.

Here's a breakdown of the key aspects:

*   **Core Principles:**
    *   **Proximity:** Computation happens near the data source (e.g., sensors, devices).
    *   **Real-time Process

## Interactive Testing Function
For testing different queries and evaluating routing behavior

In [27]:
# Interactive query function with debug info
def ask_question(question, show_debug=True):
    print(f"Question: {question}")
    response = graph.invoke({"messages": [HumanMessage(content=question)]}, config=config)
    print("\nResponse:")
    for msg in response['messages']:
        if msg.type == "ai":
            print(msg.content)
    
    # Show debug info if requested
    if show_debug and 'debug_info' in response:
        print("\nDebug Info:")
        for k, v in response.get('debug_info', {}).items():
            print(f"{k}: {v}")
    return response

In [28]:
# Test a course material query
ask_question("What does GDPR stand for, and which security document mentions it?")

Question: What does GDPR stand for, and which security document mentions it?
[DEBUG] Router received query: What does GDPR stand for, and which security document mentions it?
[DEBUG] Router decision: web_browser_query
[DEBUG] Processing in web_browser_query node
[DEBUG] Router decision: web_browser_query
[DEBUG] Processing in web_browser_query node
[DEBUG] Created web query: "GDPR definition and security documents mentioning GDPR"
[DEBUG] Processing in browser_node
[DEBUG] No URL detected - using browsing agent for general search
[DEBUG] Created web query: "GDPR definition and security documents mentioning GDPR"
[DEBUG] Processing in browser_node
[DEBUG] No URL detected - using browsing agent for general search
[DEBUG] Browser node response length: 553
[DEBUG] Processing in study_assistant node (goto=web_browser_query)
[DEBUG] Using web browsing response
[DEBUG] Browser node response length: 553
[DEBUG] Processing in study_assistant node (goto=web_browser_query)
[DEBUG] Using web brows

{'messages': [HumanMessage(content='https://en.wikipedia.org/wiki/Artificial_intelligence scrape this page about AI', additional_kwargs={}, response_metadata={}, id='9e7390b2-d777-4351-978b-3b3252a880dc'),
  AIMessage(content='I have scraped the Wikipedia page on Artificial Intelligence and extracted the table of contents. The contents include:\n\n1.  Goals\n    *   Reasoning and problem-solving\n    *   Knowledge representation\n    *   Planning and decision-making\n    *   Learning\n    *   Natural language processing\n    *   Perception\n    *   Social intelligence\n    *   General intelligence\n2.  Techniques\n    *   Search and optimization\n        *   State space search\n        *   Local search\n    *   Logic\n    *   Probabilistic methods for uncertain reasoning\n    *   Classifiers and statistical learning methods\n    *   Artificial neural networks\n    *   Deep learning\n    *   GPT\n    *   Hardware and software\n3.  Applications\n    *   Health and medicine\n    *   Games

In [29]:
# Test a web search query
ask_question("What are the most recent developments in quantum computing?")

Question: What are the most recent developments in quantum computing?
[DEBUG] Router received query: What are the most recent developments in quantum computing?
[DEBUG] Router decision: web_browser_query
[DEBUG] Processing in web_browser_query node
[DEBUG] Router decision: web_browser_query
[DEBUG] Processing in web_browser_query node
[DEBUG] Created web query: "Recent developments in quantum computing"
[DEBUG] Processing in browser_node
[DEBUG] No URL detected - using browsing agent for general search
[DEBUG] Created web query: "Recent developments in quantum computing"
[DEBUG] Processing in browser_node
[DEBUG] No URL detected - using browsing agent for general search
[DEBUG] Browser node response length: 452
[DEBUG] Processing in study_assistant node (goto=web_browser_query)
[DEBUG] Using web browsing response
[DEBUG] Browser node response length: 452
[DEBUG] Processing in study_assistant node (goto=web_browser_query)
[DEBUG] Using web browsing response

Response:
I have scraped the

{'messages': [HumanMessage(content='https://en.wikipedia.org/wiki/Artificial_intelligence scrape this page about AI', additional_kwargs={}, response_metadata={}, id='9e7390b2-d777-4351-978b-3b3252a880dc'),
  AIMessage(content='I have scraped the Wikipedia page on Artificial Intelligence and extracted the table of contents. The contents include:\n\n1.  Goals\n    *   Reasoning and problem-solving\n    *   Knowledge representation\n    *   Planning and decision-making\n    *   Learning\n    *   Natural language processing\n    *   Perception\n    *   Social intelligence\n    *   General intelligence\n2.  Techniques\n    *   Search and optimization\n        *   State space search\n        *   Local search\n    *   Logic\n    *   Probabilistic methods for uncertain reasoning\n    *   Classifiers and statistical learning methods\n    *   Artificial neural networks\n    *   Deep learning\n    *   GPT\n    *   Hardware and software\n3.  Applications\n    *   Health and medicine\n    *   Games

In [30]:
# Test URL scraping with specific URL
ask_question("Can you scrape and summarize the content from https://github.com/langchain-ai/langgraph")

Question: Can you scrape and summarize the content from https://github.com/langchain-ai/langgraph
[DEBUG] Router received query: Can you scrape and summarize the content from https://github.com/langchain-ai/langgraph
[DEBUG] Router decision: web_browser_query
[DEBUG] Processing in web_browser_query node
[DEBUG] Router decision: web_browser_query
[DEBUG] Processing in web_browser_query node
[DEBUG] Created web query: scrape and summarize content from https://github.com/langchain-ai/langgraph
[DEBUG] Processing in browser_node
[DEBUG] URL detected in query - attempting to scrape
[DEBUG] Scraping URL: https://github.com/langchain-ai/langgraph
[DEBUG] Created web query: scrape and summarize content from https://github.com/langchain-ai/langgraph
[DEBUG] Processing in browser_node
[DEBUG] URL detected in query - attempting to scrape
[DEBUG] Scraping URL: https://github.com/langchain-ai/langgraph
[DEBUG] Browser node response length: 2135
[DEBUG] Processing in study_assistant node (goto=web_b

{'messages': [HumanMessage(content='https://en.wikipedia.org/wiki/Artificial_intelligence scrape this page about AI', additional_kwargs={}, response_metadata={}, id='9e7390b2-d777-4351-978b-3b3252a880dc'),
  AIMessage(content='I have scraped the Wikipedia page on Artificial Intelligence and extracted the table of contents. The contents include:\n\n1.  Goals\n    *   Reasoning and problem-solving\n    *   Knowledge representation\n    *   Planning and decision-making\n    *   Learning\n    *   Natural language processing\n    *   Perception\n    *   Social intelligence\n    *   General intelligence\n2.  Techniques\n    *   Search and optimization\n        *   State space search\n        *   Local search\n    *   Logic\n    *   Probabilistic methods for uncertain reasoning\n    *   Classifiers and statistical learning methods\n    *   Artificial neural networks\n    *   Deep learning\n    *   GPT\n    *   Hardware and software\n3.  Applications\n    *   Health and medicine\n    *   Games